# Code Hotspot-Analyse

## Datensammlung

### Metrik 1: Anzahl der Zeilen pro Quellcodedatei

In [1]:
!find spring-framework-petclinic/ -name '*.java' | xargs wc -l | head -n-1 > lines.txt

In [2]:
import pandas as pd

lines = pd.read_csv("lines.txt", sep=r'\s+', names=['lines', 'filepath'])
lines['filepath'] = lines['filepath'].str.replace("spring-framework-petclinic/", "")
lines = lines.set_index('filepath')
lines.head()

,lines
filepath,
.mvn/wrapper/MavenWrapperDownloader.java,117
src/main/java/org/springframework/samples/petclinic/model/BaseEntity.java,47
src/main/java/org/springframework/samples/petclinic/model/NamedEntity.java,48
src/main/java/org/springframework/samples/petclinic/model/Owner.java,153
src/main/java/org/springframework/samples/petclinic/model/package-info.java,5


### Metrik 2: Änderungsinformationen

In [3]:
!git -C spring-framework-petclinic log --name-only --pretty=format: -- *.java > changes.txt

In [4]:
each_change_per_file = pd.read_csv("changes.txt", names=["changes"])
changes = pd.DataFrame(each_change_per_file['changes'].value_counts())
changes.head()

,changes
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcOwnerRepositoryImpl.java,27
src/main/java/org/springframework/samples/petclinic/web/OwnerController.java,23
src/main/java/org/springframework/samples/petclinic/web/PetController.java,22
src/test/java/org/springframework/samples/petclinic/service/AbstractClinicServiceTests.java,21
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcVetRepositoryImpl.java,20


### Metrik 3: Hotspots

In [5]:
hotspots = changes.join(lines).dropna()
hotspots.head()

,changes,lines
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcOwnerRepositoryImpl.java,27,158.0
src/main/java/org/springframework/samples/petclinic/web/OwnerController.java,23,135.0
src/main/java/org/springframework/samples/petclinic/web/PetController.java,22,113.0
src/test/java/org/springframework/samples/petclinic/service/AbstractClinicServiceTests.java,21,203.0
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcVetRepositoryImpl.java,20,88.0


## Erste Auswertungen

Zeige die Top 10 der meist geänderten Dateien an.

In [6]:
hotspots.sort_values(by="changes", ascending=False).head(10)[['changes']]

,changes
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcOwnerRepositoryImpl.java,27
src/main/java/org/springframework/samples/petclinic/web/OwnerController.java,23
src/main/java/org/springframework/samples/petclinic/web/PetController.java,22
src/test/java/org/springframework/samples/petclinic/service/AbstractClinicServiceTests.java,21
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcVetRepositoryImpl.java,20
src/main/java/org/springframework/samples/petclinic/web/VetController.java,19
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcVisitRepositoryImpl.java,19
src/main/java/org/springframework/samples/petclinic/web/VisitController.java,18
src/main/java/org/springframework/samples/petclinic/repository/jpa/JpaOwnerRepositoryImpl.java,17
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcPetRepositoryImpl.java,17


Zeige die Top 10 der größten Dateien an.

In [7]:
hotspots.sort_values(by="lines", ascending=False).head(10)[['lines']]

,lines
src/test/java/org/springframework/samples/petclinic/service/AbstractClinicServiceTests.java,203.0
src/test/java/org/springframework/samples/petclinic/web/OwnerControllerTests.java,181.0
src/main/java/org/springframework/samples/petclinic/repository/jdbc/OneToManyResultSetExtractor.java,159.0
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcOwnerRepositoryImpl.java,158.0
src/main/java/org/springframework/samples/petclinic/model/Owner.java,153.0
src/main/java/org/springframework/samples/petclinic/web/OwnerController.java,135.0
src/main/java/org/springframework/samples/petclinic/model/Visit.java,121.0
src/test/java/org/springframework/samples/petclinic/web/PetControllerTests.java,120.0
.mvn/wrapper/MavenWrapperDownloader.java,117.0
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcPetRepositoryImpl.java,116.0


Berechne und zeige Top 10 Hotspots an.

In [8]:
hotspots["rel_changes"] = hotspots['changes'] / hotspots['changes'].max()
hotspots["rel_lines"] = hotspots['lines'] / hotspots['lines'].max()
hotspots['rel_hotspot'] = hotspots["rel_changes"] + hotspots["rel_lines"]
hotspots.sort_values(by="rel_hotspot", ascending=False)[["rel_hotspot"]].head(10)

,rel_hotspot
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcOwnerRepositoryImpl.java,1.778325
src/test/java/org/springframework/samples/petclinic/service/AbstractClinicServiceTests.java,1.777778
src/main/java/org/springframework/samples/petclinic/web/OwnerController.java,1.516876
src/main/java/org/springframework/samples/petclinic/web/PetController.java,1.371465
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcVisitRepositoryImpl.java,1.211093
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcPetRepositoryImpl.java,1.201058
src/main/java/org/springframework/samples/petclinic/repository/jdbc/JdbcVetRepositoryImpl.java,1.174238
src/main/java/org/springframework/samples/petclinic/web/VisitController.java,1.129721
src/main/java/org/springframework/samples/petclinic/model/Owner.java,1.087028
src/main/java/org/springframework/samples/petclinic/web/VetController.java,1.078088


## Visualisierung

Hinweis: die einzelnen Code-Blöcke sind nur Implementierungsdetails zur Ausgabe der Visualisierung und daher nicht leserelevant.

In [9]:
from matplotlib import cm
from matplotlib.colors import rgb2hex
import json
import os


# Daten für Visualisierung aufbereiten
def create_plot_data(df, color_column_name, size_column_name, seperator):
    plot_data = pd.DataFrame(index=df.index)
    plot_data['value_for_color'] = df[color_column_name]
    plot_data['ratio_for_color'] = plot_data['value_for_color'] / plot_data['value_for_color'].max()
    plot_data['color'] = plot_data['ratio_for_color'].apply(lambda x : rgb2hex(cm.coolwarm(x)))
    plot_data['size'] = df[size_column_name]
    plot_data[['path', 'name']] = df.index.str.rsplit(seperator, n=1).to_list()
    plot_data['path_list'] = plot_data['path'].str.split(seperator)
    return plot_data


# in D3-Datenformat umwandeln
def create_flare_json(df):

    json_data = {'name': 'flare', 'children': []}

    for _, series in df.iterrows():
        hierarchical_data = series['path_list']

        children = json_data['children']
        for part in hierarchical_data:
            entry = next((child for child in children if child.get('name', '') == part), None)
            if not entry:
                entry = {'name': part, 'children': []}
                children.append(entry)
            children = entry['children']

        children.append({
            'name': f"{series['name']} ({series['size']} [{series['value_for_color']}])",
            'size': series['size'],
            'color': series['color']
        })

    return json_data


# Template mit Daten füttern
def create_hotspot_file(hotspots, color_column_name, size_column_name, separator):
    json_data = create_flare_json(create_plot_data(hotspots, color_column_name, size_column_name,separator))
            
    with open("d3/template_hierarchical_d3_inline.html") as html_template:
        html = html_template.read().replace("###FLARE_JSON###", str(json_data))
        
        os.makedirs("output", exist_ok=True)
        with open(f'hotspots.html', mode='w') as html_out:
            html_out.write(html)

### Erzeuge Hotspot-Visualisierung

In [10]:
create_hotspot_file(hotspots, "changes", "lines", "/")